In [206]:
# Прогноз результатов матчей на основе TrueSkill

Загружаем нужные библиотеки.

SyntaxError: invalid syntax (<ipython-input-206-e27a0f94b8b0>, line 3)

In [207]:
import pandas as pd
import trueskill

from math import sqrt, log
from trueskill import BETA
from trueskill.backends import cdf

Загружаем информацию про результаты игр, скачанную с http://dotascience.com/ и сразу сортируем по Match_ID.

In [6]:
matches_info = pd.read_csv('selected_team_matches.csv').sort(['match_id'])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


Инициализируем параметры Trueskill используя параметры по умолчанию, кроме вероятность ничейного результата, у нас не бывает ничьих.

In [208]:
trueskill.setup(draw_probability=0)

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.0%)

Создадим словарик, где будем хранить рейтинги. И обучим нашу систему. Для каждого матча мы определим победителя и пересчитаем рейтинги комманд. Если команда раньше не встречалась задаем ей рейтинг по умочанию. В нашей простейшей модели мы считаем, что каждая команда это один виртуальный игрок.

In [209]:
Rates = {}
for index, row in matches_info.iterrows():
    team1 = row['radiant']
    team2 = row['dire']
    if team1 not in Rates:
        Rates[team1] = trueskill.Rating()
    if team2 not in Rates:
        Rates[team2] = trueskill.Rating()
    if row['winner'] == 'RADIANT':
        Rates[team1], Rates[team2] = trueskill.rate_1vs1(Rates[team1], Rates[team2])
    else:
        Rates[team2], Rates[team1] = trueskill.rate_1vs1(Rates[team2], Rates[team1])
        

NameError: name 'matches_info' is not defined

Теперь мы можем, например, насладится рейтингом команты Virtus.pro, которая скрывается под тагом 'VP 2' 

In [210]:
Rates['VP 2']

KeyError: 'VP 2'

Попробуем оценить вероятность победы Virtus.pro, например, в матче с Evil Genius.
К сожалению в библиотеке trueskill, нет возможности оценить вероятность, но мы же знаем из описания,
что рейтинги тут нормально распределенные. Так что сделаем функцию, которая оценит вероятность, 
что сила игры первой команды, будет больше чем у второй.

In [211]:
def win_probability(player_rating, opponent_rating):
    delta_mu = player_rating.mu - opponent_rating.mu
    denom = sqrt(2 * (BETA * BETA) + pow(player_rating.sigma, 2) + pow(opponent_rating.sigma, 2))
    return cdf(delta_mu / denom)

Ура, теперь мы можем оценить шансы.

In [212]:
win_probability(Rates['VP 2'], Rates['EG'])

KeyError: 'VP 2'

In [213]:
heroes_csv = pd.read_csv('heroes.csv')
print(heroes_csv)


      id       localized_name                      name
0      1            Anti-Mage                  antimage
1      2                  Axe                       axe
2      3                 Bane                      bane
3      4          Bloodseeker               bloodseeker
4      5       Crystal Maiden            crystal_maiden
5      6          Drow Ranger               drow_ranger
6      7          Earthshaker               earthshaker
7      8           Juggernaut                juggernaut
8      9               Mirana                    mirana
9     11         Shadow Fiend                 nevermore
10    10            Morphling                 morphling
11    12       Phantom Lancer            phantom_lancer
12    13                 Puck                      puck
13    14                Pudge                     pudge
14    15                Razor                     razor
15    16            Sand King                 sand_king
16    17         Storm Spirit              storm

In [214]:
import requests
api_key = 'E80C0BAEE50452950C911FA18788536F'

In [215]:
#resp = requests.get('http://alchemist.dotascience.com/api/matches', headers={'Key': api_key})
#json_object = json.load(resp)

In [216]:
heroes_dict_name_by_id = {}
heroes_dict_id_by_name = {}
for l in heroes_csv.iterrows():
    #print(l)
    heroes_dict_name_by_id[l[1]['id']] = l[1]['localized_name']
    heroes_dict_id_by_name[l[1]['localized_name']] = l[1]['id']
    
#heroes_dict

In [217]:
import json
X_train = []
Y_train = []
iter_num = 0
bad_count = 0
with open('selected_team_match_info.jsonlines', 'r') as matches_file:
    for line in matches_file:
        #print('iteration', iter_num)
        #print(line)
        match = json.loads(line)
        #print(match['match_id'])
        if (not 'picks_bans' in match) or (not 'radiant_win' in match):
            bad_count += 1
            continue
        radiant_win = match['radiant_win']
        
        list_radiant = []
        list_dire = []
        
        for pick_info in match['picks_bans']:
            if pick_info['is_pick'] == False:
                #print('BAN')
                continue
            #else:
                #print('PICK')
            side_radiant_flag = (pick_info['team'] == 0)
            hero_id = pick_info['hero_id']
            hero_name = heroes_dict_name_by_id[hero_id]
            if side_radiant_flag:
                list_radiant.append(hero_id)
            else:
                list_dire.append(hero_id)
            #print(pick_info)
        #print(match['picks_bans'])
        list_radiant = sorted(list_radiant)
        list_dire = sorted(list_dire)
        
        #print('Radiant', list_radiant)
        #print('Dire', list_dire)
        #print('flag', radiant_win)
        X_train.append(list_radiant + list_dire)
        Y_train.append(radiant_win)
        iter_num += 1
        # Обработка матча
        #
#print(len(X_train))


In [218]:
print(bad_count, len(X_train))

(216, 2908)


In [219]:
from sklearn.neighbors import KDTree
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [271]:

class KDNN():
    def __init__(self):
        pass
    def fit(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train
        return self
    def predict(self, X_test, k):
        num_test_objects = X_test.shape[0]
                #print("num_test_objects = " + str(num_test_objects))
        y_pred = np.zeros(num_test_objects)
        for i in range(num_test_objects):
            dist = []
            for train_elem in range(len(self.X_train)):
                d = 0
                for j in range(len(self.X_train[train_elem])):
                    if self.X_train[train_elem][j] != X_test[i][j]:
                        d += 1
                dist.append(d)
                #print("iteration " + str(i))
            dist = np.array(dist)
            indexes = dist.argsort()[:k]
            #print("dist:")
            #print(dist)
            sum = 0
            #assert len(ind[0]) == k + 1
            #assert len(dist) == k + 1
            for index in indexes:
                #print("j = " + str(j))
                sum += self.Y_train[index]
            y_pred[i] = sum * 1.0 / k
        #print("len of y_pred = " + str(len(y_pred)), y_pred)
        assert len(y_pred) == num_test_objects
        return y_pred

In [264]:
def RSS(right_ans, predicted):
    result = 0
    for j in range(len(right_ans)):
        result += (predicted[j] - right_ans[j]) ** 2
    return result

In [265]:
def cross_validation(X, y, k, cv_fold=5):
    scores = []
    n = len(X)
    rnd_parts = np.random.randint(0, cv_fold, size=n)
    #print(rnd_parts)
    parts_x = [[] for x in range(cv_fold)]
    #print(parts_x)
    parts_y = [[] for x in range(cv_fold)]
    for j in range(n):
        parts_x[rnd_parts[j]].append(X[j])
        parts_y[rnd_parts[j]].append(y[j])
    for test_part in range(cv_fold):
        X_train = []
        Y_train = []
        for part in range(cv_fold):
            if test_part != part:
                X_train += parts_x[part]
                Y_train += parts_y[part]
        knn = KDNN()
        knn = knn.fit(np.asarray(X_train), np.asarray(Y_train))
        predicted = knn.predict(np.asarray(parts_x[test_part]), k)
        #assert len(predicted) == len(parts_x[test_part]), str(len(predicted)) + "!= " + str(len(parts_x[test_part]))
        #print("test x len = " + str(len(parts_x[test_part])))
        #plot_image(np.asarray(parts_x[test_part][0]))
        #print(parts_x[test_part])
        #print("y_pred len = " + str(len(predicted)))
        #print(predicted)
        rss = RSS(parts_y[test_part], predicted)
        #print("acc = " + str(acc))
        scores.append(rss)
    return np.mean(scores)


In [266]:
#make main KNN
knn = KDNN()
knn = knn.fit(X_train, Y_train)

In [267]:
def test_model_1ex(radiant_names, dire_names, k):
    radiant = []
    dire = []
    for name in radiant_names:
        radiant.append(heroes_dict_id_by_name[name])
    for name in dire_names:
        dire.append(heroes_dict_id_by_name[name])
    radiant = sorted(radiant)
    dire = sorted(dire)
    X_test = [radiant + dire]
    predict_res = knn.predict(np.asarray(X_test), k)
    return predict_res

In [ ]:
import bz2
import json
import pprint
#print(heroes_csv[1])
calculated_matches = {}
with bz2.BZ2File('shanghai_live_league_games.jsonlines.bz2') as f: 
    for line in f:
        record = json.loads(line)
        #print(record)
        if 'scoreboard' in record:
            duration = record['scoreboard']['duration']
        elif 'duration' in record:
            duration = record['duration']
        else:
            continue
        if duration < 300:
            continue
        match_id = int(record['match_id'])
        #print('match id   ' + str(match_id))
        if match_id in calculated_matches:
            continue
        calculated_matches[match_id] = 1
        print("radiant name " + str(record['radiant_team']['team_name']))
        print("dire name " + str(record['dire_team']['team_name']))
        
        
        radiant_list = []
        dire_list = []
        #pprint.pprint(record)
        match_id = record['match_id']
        for pl in record['players']:
            if pl['team'] <= 1:
                hero_id = pl['hero_id']
                assert hero_id != 0
                if pl['team'] == 0:
                    radiant_list.append(hero_id)
                else:
                    dire_list.append(hero_id)
                #print(heroes_dict_name_by_id[pl['hero_id']], pl['name'])
                #print(pl['hero_id'], pl['name'])   
        X_test = radiant_list + dire_list
        #print('X_test')
        #for j in X_test:
        #    print(heroes_dict_name_by_id[j])
        y_pred = knn.predict(np.asarray([X_test]), 50)
        print("predicted: " + str(y_pred))
        #break
        # process record

In [269]:
radiant_names = ['Earth Spirit', 'Lion', 'Invoker', 'Slark', 'Beastmaster']
dire_names = ['Rubick', 'Death Prophet', 'Tusk', 'Nature\'s Prophet', 'Outworld Devourer']
test_model_1ex(radiant_names, dire_names, 15)

array([ 0.4])

In [ ]:
radiant_names = ['Earth Spirit', 'Lion', 'Invoker', 'Slark', 'Beastmaster']
dire_names = ['Rubick', 'Death Prophet', 'Tusk', 'Nature\'s Prophet', 'Outworld Devourer']
test_model_1ex(radiant_names, dire_names)

In [274]:
#print(len(X_train))

2908


In [273]:
import matplotlib.pyplot as plt
%matplotlib inline
all_res = {}
for k in range(1, 20):
    cv_res = cross_validation(X_train, Y_train, k)
    print("k = " + str(k) + " cv_res = " + str(cv_res))
    all_res[k] = cv_res
plt.scatter(all_res.keys(), all_res.values())    
#150 previous    
    

KeyboardInterrupt: 

Система сказала, что шансы почти равные. Теперь полученную оценку можно загрузить на сайт в качетве прогноза. 
Как это сделать расказанно [тут](https://github.com/romovpa/dotascience-hackathon/blob/master/README.md).